---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [7]:
df = pd.DataFrame(df, columns=['lines'])


In [8]:
#
import re
def extract_date(line):
    """Returns an extracted date from a line of text text"""
    
   
    pattern_1 = '((\d{1,2}\s?)?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\.\,]?)) \d{4,4}'
    pattern_2 = '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\.,]?[\s-](\d{1,2}[th|st|rd|nd]?,?[\s-])?\d{4,4}'
    pattern_3 = '((\d{,2}[/-])(\d{,2}[/-])\d{2,4})'
    pattern_4 = '((\d{,2}/)\d{2,4})|\d{4,4}'
    
    patterns = [pattern_1,pattern_2,pattern_3,pattern_4]
    for pattern in patterns:
        match = re.search(pattern, line)
        if match:
            return match.group()


In [9]:
df['Extracted_date'] = df['lines'].apply(extract_date)

In [10]:
def separate_date(date):
    """ Returns a string of date, that can be coverted to a timeStammp,
        from extracted date from a text 
    """
    
    
    month_list_2 = ['Janaury','February','March','April','May','June',
                    'July','August','September','October','November','December']
    
    month_list_1 = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
                    
    months_1 = {month : val+1 for val, month in enumerate(month_list_1)}
    
    months_2 = {month : val+1 for val, month in enumerate(month_list_2)}
    
    dates = re.findall(r'\d{1,4}',date)
    if len(dates) == 3:
        
        #if re.search(r'\d{1,2}-\d{1,2}-\d{2,4}', date):
         #   day, month, year = dates[0], dates[1], dates[2]  
        
        day, month, year = dates[1], dates[0], dates[2]
        
    elif len(dates) == 2:
        found = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*',date)
        if found:
            day, month, year = dates[0], found.group(), dates[1]
        else:
            day, month, year = '01', dates[0], dates[1]
            
    elif len(dates) == 1:
        found = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*',date)
        if found:
            day, month, year = '01', found.group(), dates[0]
        else:
            day, month, year = '01', 'Jan', dates[0]
        
        
    # Checking and correcting year format
    
    if len(year) == 2:
        tens = year
        year = '19'+tens
        
    #checking and uniforming month label
    
    if month.isdigit() == False:
        try:
            month = months_1[month]
        
        except KeyError:
            month = months_1[month[:3]]
    elif len(month) > 2:
        month = month.lstrip('0')
    # Filling in missing months and day
    time = '-'.join([str(day),str(month),str(year)])
    return time

In [11]:
df['Time'] = df['Extracted_date'].apply(separate_date)

In [12]:
df['Time'] = pd.to_datetime(df['Time'], dayfirst = True, format='%d-%m-%Y')

In [189]:
#df.head(20)

In [190]:
df['Day'], df['Month'], df['Year'] = df['Time'].apply(lambda time: time.day), df['Time'].apply(lambda time: time.month), df['Time'].apply(lambda time: time.year)

In [191]:
df2 = df.copy()

In [192]:
def date_sorter():
    
    index = df.index
    sorted_df = df.sort_values('Time')
    index = sorted_df.index
    
    series = pd.Series(index)
    
    return series

In [196]:
date_sorter()